In [3]:
%run Container.ipynb
%run GameState.ipynb

from numpy.random import choice

In [203]:
class Shop(GameState):
    """
    Inherits process_triggers() and clean() from GameState class.
    """
    max_size = 7
    max_pet_slots = {1:3, 2:3, 3:3, 4:3, 5:4, 6:4, 7:4, 8:4}
    max_food_slots = {1:1, 2:1}
    tier_dict = {1:1, 2:1, 3:2, 4:2, 5:3, 6:3, 7:4, 8:4, 9:5, 10:5}
    lives_lost_dict = {1:1, 2:1, 3:2, 4:2}
    
    def __init__(self, team1 = Team([]), 
                 animal_dict = pack1_animal_dict, food_dict = pack1_food_dict,
                 verbose = False):
        assert isinstance(team1, Team), f'team1, {team1}, must be a Team object'
        self.team1 = team1
        self.team2 = ShopTeam(animal_dict)
        self.foods = ShopFoods(food_dict)
        self.animal_dict = animal_dict
        
        self.gold = 10
        self.lives = 10
        self.n_wins = 0
        self.current_turn = 1
        self.lost_last_battle = False
        self.current_tier = 1
        self.lives_lost = 1
        self.verbose = verbose
        
    def __repr__(self):
        return str(self.team1) + '\n\n' + str(self.team2) + '\n' + str(self.foods)
    
    def __str__(self):
        data = {0: ('Gold:', self.gold), 1:('Lives:', self.lives), 
                2: ('Wins:', self.n_wins), 3: ('Turn:', self.current_turn)}
        output = ''
        for i in range(max(len(self.team1), len(self.team2) + len(self.foods))):
            pet1 = self.team1[i] if i < len(self.team1) else ''
            pet2 = (self.team2.pets + self.foods.pets)[i] if i < len(self.team2) + len(self.foods.pets) else ''
            sep = 'Shop:' if i == 0 else ''
            output += f'{pet1!s:50} {sep:5} {pet2!s:30} {data.get(i, ("",""))[0]:6} {data.get(i, ("",""))[1]}\n'
        return output
    
    @property
    def current_turn(self):
        return self._current_turn
    @current_turn.setter
    def current_turn(self, new_val):
        self._current_turn = new_val
        self.current_tier = Shop.tier_dict.get(self._current_turn, 6)
        self.lives_lost = Shop.lives_lost_dict.get(self._current_turn, 3)
        
    def buy_food(self, from_ind, to_ind=0):
        """
        Method that applies a food's effect onto a pet
        """
        assert from_ind in range(len(self.foods)), f'buy_food got a bad from_ind: {from_ind}'
        assert to_ind in range(len(self.team1)), f'buy_food got a bad to_ind: {to_ind}'
        
        # apply food ability
        food = self.foods.drop(from_ind)
        self.gold -= food.cost
        assert self.gold >= 0, 'insufficient funds'
        cat_mult = 1 + sum([pet.lvl if pet.species == 'Cat' else 0 for pet in self.team1])
        food.use([self.team1[to_ind]], self, cat_mult=cat_mult, verbose=self.verbose)
        
        self.trigger_abilities('BuyFood')
        if food.name == 'Apple':
            self.team1[to_ind].trigger_dict['EatsApple'] = 1
        self.process_triggers(self.verbose)
    
    
    def buy_pet(self, from_ind, to_ind=None, do_merge=False):
        """
        Method that moves a pet from the shop to the team.
        """
        if to_ind is None:
            to_ind = len(self.team1)
        assert from_ind < len(self.team2), f'buy_pet received a bad from_ind: {from_ind}'
        assert to_ind <= len(self.team1), f'buy_pet received a bad to_ind: {to_ind}'
        assert not do_merge or to_ind < len(self.team1), f'buy_pet received a bad to_ind: {to_ind}'
        assert do_merge or len(self.team1) < Team.max_size, f'No space in team {self.team1} to buy pet without merging!'
        assert not do_merge or self.team2[from_ind].species == self.team1[to_ind].species, \
            f'Cannot merge {self.team2[from_ind].species} onto {self.team1[to_ind].species}!'
            
        to_buy = self.team2.drop(from_ind)
        self.gold -= to_buy.cost
        assert self.gold >= 0, 'insufficient funds'
        new_pet = to_buy.convert_to_pet()
        verb = 'merging' if do_merge else 'inserting'
        if self.verbose: print(f'Buying and {verb} {new_pet.species} into index {to_ind}.')
            
        if new_pet.tier == 1:
            self.trigger_abilities('BuyTier1Pet')
            
        if do_merge:
            self.team1[to_ind].trigger_dict['Buy'] = 1
            self.merge(self.team1[to_ind], new_pet)
        else:
            self.team1.insert(new_pet, to_ind)
            new_pet.trigger_dict['Buy'] = 1
            new_pet.trigger_dict['Summon'] = 1
            self.process_triggers(self.verbose)
        
    def sell_pet(self, ind):
        assert ind in range(len(self.team1)), f'sell_pet received a bad ind: {ind}'
        
        to_sell = self.team1[ind]
        if self.verbose:
            print(f'Selling {to_sell!s}.')
        to_sell.trigger_dict['Sell'] = 1
        self.process_triggers(self.verbose)
        
        self.gold += to_sell.lvl
        self.team1.drop(ind)
        
        if self.verbose:
            print('Done:')
            print(self)
        
    def roll(self, free_roll=False, trigger_abilities=True):
        if not free_roll:
            self.gold -= 1
            assert self.gold >= 0, 'insufficient funds'
            
        if self.verbose: print('Rolled shop.')
            
        # math with frozen pets/foods to determine how many new items in shop
        n_frozen_foods = len([food for food in self.foods if food.is_frozen])
        n_frozen_pets = len([pet for pet in self.team2 if pet.is_frozen])
        
        n_max_pets = Shop.max_pet_slots.get(self.current_turn, 5)
        n_max_foods = Shop.max_food_slots.get(self.current_turn, 2)
        
        n_pets = min(n_max_pets, 7-n_frozen_foods)
        n_foods = min(n_max_foods, 7-n_frozen_pets)
        
        # roll
        self.team2.restock(self.current_tier, n_pets)
        self.foods.restock(self.current_tier, n_foods)
        
        if trigger_abilities:
            self.trigger_abilities('Roll')
            self.process_triggers(self.verbose)
        
    def merge(self, base, fodder):
        """
        Helper function that does stat and level calculations for merging pets.
        """
        assert fodder.lvl in [1, 2], 'shop.merge got a fodder pet with an incompatible level'
        assert base.lvl in [1, 2], 'shop.merge got a base pet with an incompatible level'
        assert fodder.species == base.species, f'shop.merge got two different species: {fodder.species} -> {base.species}'
        
        lvl_to_exp_dict = {1:0, 2:2}
        base_exp = lvl_to_exp_dict[base.lvl] + base.exp
        fodder_exp = lvl_to_exp_dict[fodder.lvl] + fodder.exp + 1
        base.exp += fodder_exp
        
        # add a bonus pet if level-up occured
        if base.trigger_dict['LevelUp'] > 0:
            if len(self.team2) + len(self.foods) < Shop.max_size:
                self.team2.add_pet_by_tier(min(6, self.current_tier + 1))
        
        # perm stats are higher stat + lower exp
        base.atk = max(base.atk, fodder.atk) + fodder_exp
        base.hp = max(base.hp, fodder.hp) + fodder_exp
        
        # temp stats are higher stats
        base.temp_atk = max(base.temp_atk, fodder.temp_atk)
        base.temp_hp = max(base.temp_hp, fodder.temp_hp)
        
        # statuses prioritize base
        if base.status.name == 'NoStatus':
            base.status = fodder.status
        
        # process triggers:
        self.process_triggers(self.verbose)
        
    def combine_pets(self, from_ind, to_ind):
        assert from_ind in range(len(self.team1)), f'combine_pets got a bad from_ind: {from_ind}'
        assert to_ind in range(len(self.team1)), f'combine_pets got a bad to_ind: {to_ind}'
        return self.merge(self.team1[to_ind], self.team1.drop(from_ind))
    
    def end_turn(self):
        if self.verbose: print('Ended turn.')
        self.trigger_abilities('EndTurn')
        self.process_triggers(self.verbose)
        return self.team1
    
    def start_new_turn(self, battle_result=0):
        if self.verbose: print('Started new turn.')
            
        # process result
        if battle_result == 1:
            self.n_wins += 1
            self.lost_last_battle = False
        if battle_result == 2:
            self.lives -= self.lives_lost
            self.lost_last_battle = True
        if battle_result == 0:
            self.lost_last_battle = False
           
        # reset shop
        self.current_turn += 1
        # TODO: check this
        if self.current_turn in [3, 5]:
            self.lives_lost += 1
        self.gold = 10
        self.roll(free_roll=True, trigger_abilities=False)
        
        # reset pets
        for pet in self.team1:
            pet.temp_atk = 0
            pet.temp_hp = 0
            for abl in pet.ability:
                abl.n_charges = abl.n_charges_ref
            
        # trigger abilities
        self.trigger_abilities('StartOfTurn')
        if self.current_turn in [3, 5, 7, 9, 11]:
            self.trigger_abilities('UpgradeShopTier')
        self.process_triggers(self.verbose)
    
    # for now, just use abilities without queueing because all effects are commutative
    def trigger_abilities(self, trigger):
        used = False
        for pet in self.team1:
            for abl in pet.ability:
                if abl.trigger == trigger and abl.trigger_func.team_cond(pet, self, True):
                    used = True
                    abl.use(self, True, verbose=self.verbose)
        if self.verbose and used:
            print()
    
    def freeze_food(self, ind):
        assert ind in range(len(self.foods)), f'freeze_food got a bad ind: {ind}'
        assert not self.foods[ind].is_frozen, f'food at ind {ind} is already frozen'
        self.foods[ind].is_frozen = True
        
    def unfreeze_food(self, ind):
        assert ind in range(len(self.foods)), f'freeze_food got a bad ind: {ind}'
        assert self.foods[ind].is_frozen, f'food at ind {ind} is not frozen'
        self.foods[ind].is_frozen = False
        
    def freeze_pet(self, ind):
        assert ind in range(len(self.team2)), f'freeze_pet got a bad ind: {ind}'
        assert not self.team2[ind].is_frozen, f'pet at ind {ind} is already frozen'
        self.team2[ind].is_frozen = True
        
    def unfreeze_pet(self, ind):
        assert ind in range(len(self.team2)), f'freeze_pet got a bad ind: {ind}'
        assert self.team2[ind].is_frozen, f'pet at ind {ind} is not frozen'
        self.team2[ind].is_frozen = False
    

Available actions:
* buy pet: (7 * 5 * 2)
* sell pet: 5
* combine pets: 10
* buy food: (3 * 5)
* freeze pets: 7
* freeze foods: 3
* unfreeze pets: 7
* unfreeze foods: 3
* roll: 1
* end turn: 1

* total: 122
    
Observations:
* own team
    - species: 5 * [bool array]
    - atk/hp: 5 * [int], 5 * [int]
* shop animals:
    - species: 7 * [bool array]
    - atk/hp: 7 * [int], 7 * [int]
    - frozen: 7 * [bool]
* food
    - foods: 3 * [bool_array]
    - frozen: 3 * [bool]
* gold, lives, wins, turns: 4 * [int]

* total (bool): 5 * 

'buy_pet(0,0,0)', 'buy_pet(0,1,0)', 'buy_pet(0,2,0)',
       'buy_pet(0,3,0)', 'buy_pet(0,4,0)', 'buy_pet(1,0,0)',
       'buy_pet(1,1,0)', 'buy_pet(1,2,0)', 'buy_pet(1,3,0)',
       'buy_pet(1,4,0)', 'buy_pet(2,0,0)', 'buy_pet(2,1,0)',
       'buy_pet(2,2,0)', 'buy_pet(2,3,0)', 'buy_pet(2,4,0)',
       'buy_pet(3,0,0)', 'buy_pet(3,1,0)', 'buy_pet(3,2,0)',
       'buy_pet(3,3,0)', 'buy_pet(3,4,0)', 'buy_pet(4,0,0)',
       'buy_pet(4,1,0)', 'buy_pet(4,2,0)', 'buy_pet(4,3,0)',
       'buy_pet(4,4,0)', 'buy_pet(5,0,0)', 'buy_pet(5,1,0)',
       'buy_pet(5,2,0)', 'buy_pet(5,3,0)', 'buy_pet(5,4,0)',
       'buy_pet(6,0,0)', 'buy_pet(6,1,0)', 'buy_pet(6,2,0)',
       'buy_pet(6,3,0)', 'buy_pet(6,4,0)'
       
'buy_pet(0,0,1)',
       'buy_pet(0,1,1)', 'buy_pet(0,2,1)', 'buy_pet(0,3,1)',
       'buy_pet(0,4,1)', 'buy_pet(1,0,1)', 'buy_pet(1,1,1)',
       'buy_pet(1,2,1)', 'buy_pet(1,3,1)', 'buy_pet(1,4,1)',
       'buy_pet(2,0,1)', 'buy_pet(2,1,1)', 'buy_pet(2,2,1)',
       'buy_pet(2,3,1)', 'buy_pet(2,4,1)', 'buy_pet(3,0,1)',
       'buy_pet(3,1,1)', 'buy_pet(3,2,1)', 'buy_pet(3,3,1)',
       'buy_pet(3,4,1)', 'buy_pet(4,0,1)', 'buy_pet(4,1,1)',
       'buy_pet(4,2,1)', 'buy_pet(4,3,1)', 'buy_pet(4,4,1)',
       'buy_pet(5,0,1)', 'buy_pet(5,1,1)', 'buy_pet(5,2,1)',
       'buy_pet(5,3,1)', 'buy_pet(5,4,1)', 'buy_pet(6,0,1)',
       'buy_pet(6,1,1)', 'buy_pet(6,2,1)', 'buy_pet(6,3,1)',
       'buy_pet(6,4,1)' (70)
       
'sell_pet(0)', 'sell_pet(1)', 'sell_pet(2)', 'sell_pet(3)',
       'sell_pet(4)' (5)
       
'combine_pets(0,1)', 'combine_pets(0,2)', 'combine_pets(0,3)',
       'combine_pets(0,4)', 'combine_pets(1,2)', 'combine_pets(1,3)',
       'combine_pets(1,4)', 'combine_pets(2,3)', 'combine_pets(2,4)',
       'combine_pets(3,4)' (10)
       
'buy_food(0,0)', 'buy_food(0,1)', 'buy_food(0,2)', 'buy_food(0,3)',
       'buy_food(0,4)', 'buy_food(1,0)', 'buy_food(1,1)', 'buy_food(1,2)',
       'buy_food(1,3)', 'buy_food(1,4)', 'buy_food(2,0)', 'buy_food(2,1)',
       'buy_food(2,2)', 'buy_food(2,3)', 'buy_food(2,4)' (15)
       
'freeze_pets(0)', 'freeze_pets(1)', 'freeze_pets(2)',
       'freeze_pets(3)', 'freeze_pets(4)', 'freeze_pets(5)',
       'freeze_pets(6)' (7)

'freeze_foods(0)', 'freeze_foods(1)', 'freeze_foods(2)' (3)
       
'unfreeze_pets(0)', 'unfreeze_pets(1)', 'unfreeze_pets(2)',
       'unfreeze_pets(3)', 'unfreeze_pets(4)', 'unfreeze_pets(5)',
       'unfreeze_pets(6)' (7)
       
'unfreeze_foods(0)', 'unfreeze_foods(1)', 'unfreeze_foods(2)' (3)

roll (1)

end_turn (1)

In [20]:
np.array([f'freeze_foods({i})' for i in range(3)])

array(['freeze_foods(0)', 'freeze_foods(1)', 'freeze_foods(2)'],
      dtype='<U15')